In [13]:
import findspark
findspark.init()
findspark.find()

'/usr/local/opt/apache-spark/libexec'

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = (
    SparkSession
    .builder
    .appName("Exercise3App")
    .master("local[4]")

    .config("spark.dynamicAllocation.enabled", "false")
    .config("spark.sql.adaptive.enabled", "false")

    # ============== Enable Hive support for permanent storage ===========
    .enableHiveSupport()

    .getOrCreate()
)
sc = spark.sparkContext
spark

In [15]:
from IPython.display import *
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [16]:
### Provider a file that contains (diner_id, order_id), write a spark example (scala or python) to output into a DataFrame and execute Spark SQL to get (diner_id, order_count)

dinersByBrandDF = (
                    spark
                        .read
                        .option("header", "true")
                        .option("inferSchema", "true")
                        .csv("DinersByBrand.csv")
)
dinersByBrandDF.printSchema()

root
 |-- diner_id: string (nullable = true)
 |-- brand: string (nullable = true)



In [17]:
dinersByBrandDF.show()

+--------+--------+
|diner_id|   brand|
+--------+--------+
|      d1| grubhub|
|      d2|seamless|
|      d3| grubhub|
|      d4| grubhub|
+--------+--------+



In [18]:
dinersByBrandDF.createOrReplaceTempView("DinersByBrand")

spark.sql("""
    SELECT brand, count(*) as number_of_diners FROM DinersByBrand group by brand
""").show()

+--------+----------------+
|   brand|number_of_diners|
+--------+----------------+
| grubhub|               3|
|seamless|               1|
+--------+----------------+



In [19]:
aggDinersByBrandDF = dinersByBrandDF.groupBy("brand").count()
aggDinersByBrandDF.show()

+--------+-----+
|   brand|count|
+--------+-----+
| grubhub|    3|
|seamless|    1|
+--------+-----+



In [21]:
spark.sql("""

CREATE DATABASE IF NOT EXISTS Brands

""")

DataFrame[]

In [22]:
(
    aggDinersByBrandDF
    .write
    .mode("overwrite")
    .saveAsTable("Brands.DinersCount")
)